In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from operator import itemgetter, attrgetter
from PIL import Image
import glob
from tensorflow import keras
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization, AveragePooling2D
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
# import tensorflow_datasets as tfds  # pip install tensorflow-datasets
import tensorflow as tf
import logging
import numpy as np
import numpy as np
from sklearn.model_selection import train_test_split

In [6]:
# chrome_options = Options()
# chrome_options.add_argument("--headless")
driver = webdriver.Chrome('path/chromedriver')

# Собираем 1000 капч

In [7]:
i = 1000
while i>0:
    driver.get('http://www.vybory.izbirkom.ru/region/izbirkom?action=show&vrn=4274007421995&region=27&prver=0&pronetvd=0')
    time.sleep(0.5)
    with open(str(i)+'.png', 'wb') as file:
        file.write(driver.find_element_by_xpath('//*[@id="captchaImg"]').screenshot_as_png)
    i = i - 1

# Нарезаем капчу по цифрам

In [34]:
list_img = glob.glob('path/*.png')

for img in list_img:
    im = Image.open(img)
    im = im.convert("P")
    im2 = Image.new("P",im.size,255)

    im = im.convert("P")

    temp = {}

    for x in range(im.size[1]):
        for y in range(im.size[0]):
            pix = im.getpixel((y,x))
            temp[pix] = pix
            if pix != 0: 
                im2.putpixel((y,x),0)

    im2.save(img)

# обрезаем нарезанные числа

In [ ]:
for name in glob.glob('path/*.png'):
    im2 = Image.open(name)
    resized_img = im2.resize((10, 50), Image.ANTIALIAS)
    im3 = resized_img.transpose(Image.ROTATE_90)
    letters = []

    for y in range(im3.size[0]): # slice across
        for x in range(im3.size[1]): # slice down
            pix = im3.getpixel((y,x))
            if pix != 255:
                inletter = True
        if foundletter == False and inletter == True:
            foundletter = True
            start = y

        if foundletter == True and inletter == False:
            foundletter = False
            end = y
            letters.append((start,end))

        inletter=False
    print (letters)
    for letter in letters:
        im4 = im3.crop(( letter[0] , 0, letter[1],im3.size[1] ))
    im4 = im4.transpose(Image.ROTATE_270)
    im4.save(name)

# Загрузим данные

In [89]:
list_folder = ['0','1','2','3','4','5','6','7','8','9']
X_Train = []
y_train = []
for folder in list_folder:
    for name in glob.glob('/Users/aleksejkudrasov/Downloads/solving_captchas_code_examples/resize/'+folder+'/*.png'):
        im2 = Image.open(name)
        resized_img = im2.resize((10, 10), Image.ANTIALIAS)
        X_Train.append(np.array(resized_img))
        y_train.append(folder)

In [91]:
X_Train = np.array(X_Train) / 255.0
y_train = np.array(y_train)

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X_Train, y_train, test_size=0.15, random_state=42)

In [93]:
train_data = X_train.reshape(X_train.shape[0], 10*10)
test_data = X_test.reshape(X_test.shape[0], 10*10)
num_classes = 10

In [94]:
train_labels_cat = keras.utils.to_categorical(y_train, num_classes)
test_labels_cat = keras.utils.to_categorical(y_test, num_classes)

In [96]:
def mnist_make_model(image_w: int, image_h: int):
    # Neural network model
    model = Sequential()
    model.add(Dense(image_w*image_h, activation='relu', input_shape=(image_w*image_h,)))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
    return model

In [97]:
model = mnist_make_model(10,10)

In [100]:
model.fit(train_data, train_labels_cat, epochs=20, batch_size=32, verbose=1, validation_data=(test_data, test_labels_cat))

Epoch 1/20
27/27 [==============================] - 0s 2ms/step - loss: 5.6098e-09 - accuracy: 1.0000 - val_loss: 0.1056 - val_accuracy: 0.9867
Epoch 2/20
27/27 [==============================] - 0s 1ms/step - loss: 5.3294e-09 - accuracy: 1.0000 - val_loss: 0.1049 - val_accuracy: 0.9867
Epoch 3/20
27/27 [==============================] - 0s 1ms/step - loss: 5.1891e-09 - accuracy: 1.0000 - val_loss: 0.1051 - val_accuracy: 0.9867
Epoch 4/20
27/27 [==============================] - 0s 1ms/step - loss: 4.9086e-09 - accuracy: 1.0000 - val_loss: 0.1044 - val_accuracy: 0.9867
Epoch 5/20
27/27 [==============================] - 0s 1ms/step - loss: 5.3294e-09 - accuracy: 1.0000 - val_loss: 0.1049 - val_accuracy: 0.9867
Epoch 6/20
27/27 [==============================] - 0s 1ms/step - loss: 5.3294e-09 - accuracy: 1.0000 - val_loss: 0.1051 - val_accuracy: 0.9867
Epoch 7/20
27/27 [==============================] - 0s 1ms/step - loss: 5.1891e-09 - accuracy: 1.0000 - val_loss: 0.1046 - val_accuracy:

In [101]:
model.save_weights("model.h5")